In [1]:
# First, as always, we install all the packages we will use
import pandas as pd
import numpy as numpy
import json
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


Collectdone
done

## Package Plan ##

  environment location: /Users/thomasottotrankle/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         146 KB  conda-forge
    certifi-2020.4.5.2         |   py37hc8dfbb8_0         152 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         298 KB

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.1-hecc5488_0 --> 2020.4.5.2-hecda079_0
  certifi                         2020.4.5.1-py37hc8dfbb8_0 --> 2020.4.5.2-py37hc8dfbb8_0



ca-certificates-2020 | 146 KB    | ##################################### | 100% 
certifi-2020.4.5.2   | 152 KB    | ##################################### | 100% 
Preparing transaction:done
Verifying tr

In [2]:
CLIENT_ID = 'WSQG4TQ0ZC1HHFD22BPRNBH1T5WAE4LEFFI5VBZ03Y0OH2JS' # your Foursquare ID
CLIENT_SECRET = 'RIQDT2B5HKK3SMYL20MGFKFOFHALLFEABYMFHH3EITRYZTQC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

radius = 15000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WSQG4TQ0ZC1HHFD22BPRNBH1T5WAE4LEFFI5VBZ03Y0OH2JS
CLIENT_SECRET:RIQDT2B5HKK3SMYL20MGFKFOFHALLFEABYMFHH3EITRYZTQC


In [4]:
address = 'Washington, D.C.'
geolocator = Nominatim(user_agent="dc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude 

Additional package

In [5]:
import requests

In [6]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Define URL

In [8]:
url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, 'coffee', radius, LIMIT, '4d4b7105d754a06374d81259')
url2

'https://api.foursquare.com/v2/venues/explore?client_id=WSQG4TQ0ZC1HHFD22BPRNBH1T5WAE4LEFFI5VBZ03Y0OH2JS&client_secret=RIQDT2B5HKK3SMYL20MGFKFOFHALLFEABYMFHH3EITRYZTQC&ll=38.8949855,-77.0365708&v=20180605&query=coffee&radius=15000&limit=100&categoryId=4d4b7105d754a06374d81259'

## Send GET request and examine results

In [9]:
results2 = requests.get(url2).json()


In [10]:
'There are {} restraunts around Washington, D.C.'.format(len(results2['response']['groups'][0]['items']))

'There are 100 restraunts around Washington, D.C.'

### Examining data format extracted from Foursquare API.

In [11]:
items2 = results2['response']['groups'][0]['items']
items2[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a3a8e8c7918713861871716',
  'name': 'Compass Coffee',
  'location': {'address': '800 17th St NW',
   'crossStreet': 'btwn H & I St NW',
   'lat': 38.900382,
   'lng': -77.039935,
   'labeledLatLngs': [{'label': 'display',
     'lat': 38.900382,
     'lng': -77.039935}],
   'distance': 667,
   'postalCode': '20006',
   'cc': 'US',
   'city': 'Washington',
   'state': 'D.C.',
   'country': 'United States',
   'formattedAddress': ['800 17th St NW (btwn H & I St NW)',
    'Washington, D.C. 20006',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},


### List all 100 restraunts in dataframe.

In [12]:
df = json_normalize(items2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df.columns if col.startswith('venue.location.')] + ['venue.id']
df_filtered = df.loc[:, filtered_columns]

# filter the category for each row
df_filtered['venue.categories'] = df_filtered.apply(get_category_type, axis=1)

# clean columns
df_filtered.columns = [col.split('.')[-1] for col in df_filtered.columns]

df_filtered.head(34)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Swing's Coffee,Coffee Shop,640 14th St NW,at G St NW,38.898073,-77.031991,"[{'label': 'display', 'lat': 38.89807332447161...",524,20005,US,Washington,D.C.,United States,"[640 14th St NW (at G St NW), Washington, D.C....",NaN,5894e87d76b8b24a3400c1ff
1,Compass Coffee,Coffee Shop,800 17th St NW,btwn H & I St NW,38.900382,-77.039935,"[{'label': 'display', 'lat': 38.900382, 'lng':...",667,20006,US,Washington,D.C.,United States,"[800 17th St NW (btwn H & I St NW), Washington...",NaN,5a3a8e8c7918713861871716
2,A Baked Joint,Café,440 K St NW,at 5th St NW,38.902280,-77.017215,"[{'label': 'display', 'lat': 38.90228032532726...",1863,20001,US,Washington,D.C.,United States,"[440 K St NW (at 5th St NW), Washington, D.C. ...",Mount Vernon Square,5581a9a8498e9053810aa260
3,La Colombe Coffee Roasters,Coffee Shop,900 6th Street NW,NaN,38.901054,-77.020102,"[{'label': 'display', 'lat': 38.901054, 'lng':...",1578,20001,US,Washington,D.C.,United States,"[900 6th Street NW, Washington, D.C. 20001, Un...",NaN,5648c247498e93b42e8897c8
4,La Colombe Coffee Roasters,Coffee Shop,924 Blagden Alley NW,"btwn M, N, 9th & 10th St NW",38.906584,-77.024952,"[{'label': 'display', 'lat': 38.90658413373735...",1637,20001,US,Washington,D.C.,United States,"[924 Blagden Alley NW (btwn M, N, 9th & 10th S...",Logan Circle - Shaw,512f9781183f26d8006f53cb
5,Compass Coffee,Coffee Shop,1401 I St NW,at 14th St NW,38.901522,-77.032230,"[{'label': 'display', 'lat': 38.90152219803539...",819,20005,US,Washington,D.C.,United States,"[1401 I St NW (at 14th St NW), Washington, D.C...",NaN,5a3a7dde234724761e0eeb30
6,Dolcezza Gelato and Coffee,Ice Cream Shop,1418 14th St NW,at P St NW,38.909496,-77.032203,"[{'label': 'display', 'lat': 38.90949590599116...",1659,20001,US,Washington,D.C.,United States,"[1418 14th St NW (at P St NW), Washington, D.C...",NaN,53ac8f11498e7605c5f3d48a
7,Compass Coffee,Coffee Shop,1776 I St NW,at 18th St NW,38.901041,-77.041680,"[{'label': 'display', 'lat': 38.90104109868876...",806,20006,US,Washington,D.C.,United States,"[1776 I St NW (at 18th St NW), Washington, D.C...",NaN,57e2c355498e82b8e91f5166
8,Dolcezza Artisanal Gelato,Ice Cream Shop,1704 Connecticut Ave NW,at R St. NW,38.912786,-77.045628,"[{'label': 'display', 'lat': 38.912786, 'lng':...",2131,20009,US,Washington,D.C.,United States,"[1704 Connecticut Ave NW (at R St. NW), Washin...",NaN,4bc77107af07a59351317f2d
9,Baked & Wired,Bakery,1052 Thomas Jefferson St NW,at K St NW,38.903973,-77.060265,"[{'label': 'display', 'lat': 38.903973, 'lng':...",2283,20007,US,Washington,D.C.,United States,"[1052 Thomas Jefferson St NW (at K St NW), Was...",NaN,49bfffd0f964a5203c551fe3


In [13]:
df_filtered.drop(columns=['crossStreet','cc'],inplace=True)

## Lets explore all the restraunts in the area

## We want our restraunts to be a high quality fit so the average rating in the area should be good.

In [28]:
l = list(df_filtered.name)

### Lets start by looping through and gettig all the ratings for every restraunt 

In [16]:
#Lets create an empty dataframe for ratings.
df_ratings = pd.DataFrame(columns = ['Rating'])
df_ratings

,Rating


In [17]:
#Now we will create an empty list and iterate through the adding each locations rating
ratings_list = []
for i in range(len(df_filtered)):
    venue_id = df_filtered.id[i] # ID restraunt
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    try:
         ratings_list.append(result['response']['venue']['rating'])
    except:
         ratings_list.append('This venue has not been rated yet.')

In [18]:
#We add this info to our existing df_filtered
df_ratings['Rating'] = ratings_list
# Merge the two dataframes 
df_rated = pd.concat([df_filtered, df_ratings], axis=1, sort=False)
df_rated

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,city,state,country,formattedAddress,neighborhood,id,Rating
0,Swing's Coffee,Coffee Shop,640 14th St NW,38.898073,-77.031991,"[{'label': 'display', 'lat': 38.89807332447161...",524,20005,Washington,D.C.,United States,"[640 14th St NW (at G St NW), Washington, D.C....",NaN,5894e87d76b8b24a3400c1ff,8.8
1,Compass Coffee,Coffee Shop,800 17th St NW,38.900382,-77.039935,"[{'label': 'display', 'lat': 38.900382, 'lng':...",667,20006,Washington,D.C.,United States,"[800 17th St NW (btwn H & I St NW), Washington...",NaN,5a3a8e8c7918713861871716,8.8
2,A Baked Joint,Café,440 K St NW,38.902280,-77.017215,"[{'label': 'display', 'lat': 38.90228032532726...",1863,20001,Washington,D.C.,United States,"[440 K St NW (at 5th St NW), Washington, D.C. ...",Mount Vernon Square,5581a9a8498e9053810aa260,9.4
3,La Colombe Coffee Roasters,Coffee Shop,900 6th Street NW,38.901054,-77.020102,"[{'label': 'display', 'lat': 38.901054, 'lng':...",1578,20001,Washington,D.C.,United States,"[900 6th Street NW, Washington, D.C. 20001, Un...",NaN,5648c247498e93b42e8897c8,9.1
4,La Colombe Coffee Roasters,Coffee Shop,924 Blagden Alley NW,38.906584,-77.024952,"[{'label': 'display', 'lat': 38.90658413373735...",1637,20001,Washington,D.C.,United States,"[924 Blagden Alley NW (btwn M, N, 9th & 10th S...",Logan Circle - Shaw,512f9781183f26d8006f53cb,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Starbucks,Coffee Shop,"470 L'Enfant Plaza East, SW",38.883811,-77.023993,"[{'label': 'display', 'lat': 38.8838108, 'lng'...",1653,20024,Washington,D.C.,United States,"[470 L'Enfant Plaza East, SW, Washington, D.C....",NaN,4dc93901e4cdf14b8559aee4,This venue has not been rated yet.
96,Qualia Coffee,Coffee Shop,3917 Georgia Ave NW,38.939183,-77.024540,"[{'label': 'display', 'lat': 38.93918331623033...",5029,20011,Washington,D.C.,United States,[3917 Georgia Ave NW (btwn Randolph & Shepherd...,NaN,4ad51c5cf964a520ee0121e3,This venue has not been rated yet.
97,Philz Coffee,Coffee Shop,4121 Wilson Blvd Ste 102C,38.880234,-77.110107,"[{'label': 'display', 'lat': 38.88023433012077...",6579,22203,Arlington,VA,United States,"[4121 Wilson Blvd Ste 102C (Wilson Blvd), Arli...",NaN,5c54463fe55d8b002c112236,This venue has not been rated yet.
98,Krispy Kreme Doughnuts,Donut Shop,1350 Connecticut Ave NW,38.908712,-77.043208,"[{'label': 'display', 'lat': 38.9087119, 'lng'...",1632,20036,Washington,D.C.,United States,"[1350 Connecticut Ave NW, Washington, D.C. 200...",NaN,4aee9310f964a52009d421e3,This venue has not been rated yet.


### Display all 100 restraunts on map.

In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Washington, D.C.

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_rated.lat, df_rated.lng, df_rated.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

## Lets group them according to zip code and then remap

In [17]:
#data types of dataframe
df_rated.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
city                 object
state                object
country              object
formattedAddress     object
neighborhood         object
id                   object
Rating               object
dtype: object

Lets reduce it to locations that have a rating.

In [32]:
#converting the data type of Rating column from object to float
df_rated['Rating'] = pd.to_numeric(df_rated.Rating, errors='coerce')

In [33]:
df_rated

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,city,state,country,formattedAddress,neighborhood,id,Rating
0,Swing's Coffee,Coffee Shop,640 14th St NW,38.898073,-77.031991,"[{'label': 'display', 'lat': 38.89807332447161...",524,20005,Washington,D.C.,United States,"[640 14th St NW (at G St NW), Washington, D.C....",NaN,5894e87d76b8b24a3400c1ff,8.8
1,Compass Coffee,Coffee Shop,800 17th St NW,38.900382,-77.039935,"[{'label': 'display', 'lat': 38.900382, 'lng':...",667,20006,Washington,D.C.,United States,"[800 17th St NW (btwn H & I St NW), Washington...",NaN,5a3a8e8c7918713861871716,8.8
2,A Baked Joint,Café,440 K St NW,38.902280,-77.017215,"[{'label': 'display', 'lat': 38.90228032532726...",1863,20001,Washington,D.C.,United States,"[440 K St NW (at 5th St NW), Washington, D.C. ...",Mount Vernon Square,5581a9a8498e9053810aa260,9.4
3,La Colombe Coffee Roasters,Coffee Shop,900 6th Street NW,38.901054,-77.020102,"[{'label': 'display', 'lat': 38.901054, 'lng':...",1578,20001,Washington,D.C.,United States,"[900 6th Street NW, Washington, D.C. 20001, Un...",NaN,5648c247498e93b42e8897c8,9.1
4,La Colombe Coffee Roasters,Coffee Shop,924 Blagden Alley NW,38.906584,-77.024952,"[{'label': 'display', 'lat': 38.90658413373735...",1637,20001,Washington,D.C.,United States,"[924 Blagden Alley NW (btwn M, N, 9th & 10th S...",Logan Circle - Shaw,512f9781183f26d8006f53cb,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Starbucks,Coffee Shop,"470 L'Enfant Plaza East, SW",38.883811,-77.023993,"[{'label': 'display', 'lat': 38.8838108, 'lng'...",1653,20024,Washington,D.C.,United States,"[470 L'Enfant Plaza East, SW, Washington, D.C....",NaN,4dc93901e4cdf14b8559aee4,NaN
96,Qualia Coffee,Coffee Shop,3917 Georgia Ave NW,38.939183,-77.024540,"[{'label': 'display', 'lat': 38.93918331623033...",5029,20011,Washington,D.C.,United States,[3917 Georgia Ave NW (btwn Randolph & Shepherd...,NaN,4ad51c5cf964a520ee0121e3,NaN
97,Philz Coffee,Coffee Shop,4121 Wilson Blvd Ste 102C,38.880234,-77.110107,"[{'label': 'display', 'lat': 38.88023433012077...",6579,22203,Arlington,VA,United States,"[4121 Wilson Blvd Ste 102C (Wilson Blvd), Arli...",NaN,5c54463fe55d8b002c112236,NaN
98,Krispy Kreme Doughnuts,Donut Shop,1350 Connecticut Ave NW,38.908712,-77.043208,"[{'label': 'display', 'lat': 38.9087119, 'lng'...",1632,20036,Washington,D.C.,United States,"[1350 Connecticut Ave NW, Washington, D.C. 200...",NaN,4aee9310f964a52009d421e3,NaN


In [34]:
#drop unneccessary columns
df_reduced = df_rated.drop(columns='neighborhood')
df_reduced.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,city,state,country,formattedAddress,id,Rating
0,Swing's Coffee,Coffee Shop,640 14th St NW,38.898073,-77.031991,"[{'label': 'display', 'lat': 38.89807332447161...",524,20005,Washington,D.C.,United States,"[640 14th St NW (at G St NW), Washington, D.C....",5894e87d76b8b24a3400c1ff,8.8
1,Compass Coffee,Coffee Shop,800 17th St NW,38.900382,-77.039935,"[{'label': 'display', 'lat': 38.900382, 'lng':...",667,20006,Washington,D.C.,United States,"[800 17th St NW (btwn H & I St NW), Washington...",5a3a8e8c7918713861871716,8.8
2,A Baked Joint,Café,440 K St NW,38.902280,-77.017215,"[{'label': 'display', 'lat': 38.90228032532726...",1863,20001,Washington,D.C.,United States,"[440 K St NW (at 5th St NW), Washington, D.C. ...",5581a9a8498e9053810aa260,9.4
3,La Colombe Coffee Roasters,Coffee Shop,900 6th Street NW,38.901054,-77.020102,"[{'label': 'display', 'lat': 38.901054, 'lng':...",1578,20001,Washington,D.C.,United States,"[900 6th Street NW, Washington, D.C. 20001, Un...",5648c247498e93b42e8897c8,9.1
4,La Colombe Coffee Roasters,Coffee Shop,924 Blagden Alley NW,38.906584,-77.024952,"[{'label': 'display', 'lat': 38.90658413373735...",1637,20001,Washington,D.C.,United States,"[924 Blagden Alley NW (btwn M, N, 9th & 10th S...",512f9781183f26d8006f53cb,9.1


In [35]:
#drop rows without a rating
df_cut = df_reduced.dropna(subset=['Rating'])
df_cut

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,city,state,country,formattedAddress,id,Rating
0,Swing's Coffee,Coffee Shop,640 14th St NW,38.898073,-77.031991,"[{'label': 'display', 'lat': 38.89807332447161...",524,20005,Washington,D.C.,United States,"[640 14th St NW (at G St NW), Washington, D.C....",5894e87d76b8b24a3400c1ff,8.8
1,Compass Coffee,Coffee Shop,800 17th St NW,38.900382,-77.039935,"[{'label': 'display', 'lat': 38.900382, 'lng':...",667,20006,Washington,D.C.,United States,"[800 17th St NW (btwn H & I St NW), Washington...",5a3a8e8c7918713861871716,8.8
2,A Baked Joint,Café,440 K St NW,38.902280,-77.017215,"[{'label': 'display', 'lat': 38.90228032532726...",1863,20001,Washington,D.C.,United States,"[440 K St NW (at 5th St NW), Washington, D.C. ...",5581a9a8498e9053810aa260,9.4
3,La Colombe Coffee Roasters,Coffee Shop,900 6th Street NW,38.901054,-77.020102,"[{'label': 'display', 'lat': 38.901054, 'lng':...",1578,20001,Washington,D.C.,United States,"[900 6th Street NW, Washington, D.C. 20001, Un...",5648c247498e93b42e8897c8,9.1
4,La Colombe Coffee Roasters,Coffee Shop,924 Blagden Alley NW,38.906584,-77.024952,"[{'label': 'display', 'lat': 38.90658413373735...",1637,20001,Washington,D.C.,United States,"[924 Blagden Alley NW (btwn M, N, 9th & 10th S...",512f9781183f26d8006f53cb,9.1
5,Compass Coffee,Coffee Shop,1401 I St NW,38.901522,-77.032230,"[{'label': 'display', 'lat': 38.90152219803539...",819,20005,Washington,D.C.,United States,"[1401 I St NW (at 14th St NW), Washington, D.C...",5a3a7dde234724761e0eeb30,8.6
6,Dolcezza Gelato and Coffee,Ice Cream Shop,1418 14th St NW,38.909496,-77.032203,"[{'label': 'display', 'lat': 38.90949590599116...",1659,20001,Washington,D.C.,United States,"[1418 14th St NW (at P St NW), Washington, D.C...",53ac8f11498e7605c5f3d48a,9.0
7,Compass Coffee,Coffee Shop,1776 I St NW,38.901041,-77.041680,"[{'label': 'display', 'lat': 38.90104109868876...",806,20006,Washington,D.C.,United States,"[1776 I St NW (at 18th St NW), Washington, D.C...",57e2c355498e82b8e91f5166,8.6
8,Dolcezza Artisanal Gelato,Ice Cream Shop,1704 Connecticut Ave NW,38.912786,-77.045628,"[{'label': 'display', 'lat': 38.912786, 'lng':...",2131,20009,Washington,D.C.,United States,"[1704 Connecticut Ave NW (at R St. NW), Washin...",4bc77107af07a59351317f2d,9.1
9,Baked & Wired,Bakery,1052 Thomas Jefferson St NW,38.903973,-77.060265,"[{'label': 'display', 'lat': 38.903973, 'lng':...",2283,20007,Washington,D.C.,United States,"[1052 Thomas Jefferson St NW (at K St NW), Was...",49bfffd0f964a5203c551fe3,9.1


In [36]:
#list number of restraunts with ratings 
df_rated.isnull().sum(axis = 0)

name                 0
categories           0
address              0
lat                  0
lng                  0
labeledLatLngs       0
distance             0
postalCode           0
city                 0
state                0
country              0
formattedAddress     0
neighborhood        93
id                   0
Rating              50
dtype: int64

### There are 50 restraunts without ratings 

## Acquiring the zip codes for regions with high rated restraunts.

In [37]:
df_test = df_cut.groupby('postalCode')['Rating'].mean()

In [38]:
df_test

postalCode
20001    9.000000
20002    8.860000
20003    8.960000
20005    8.600000
20006    8.566667
20007    8.728571
20009    8.800000
20010    8.900000
20036    8.500000
20037    8.200000
22202    9.000000
22209    8.700000
Name: Rating, dtype: float64

In [39]:
# Lets randomly choose the top 7 zip codes.
df_test.index[0:7].values

array(['20001', '20002', '20003', '20005', '20006', '20007', '20009'],
      dtype=object)

## Get the top 7 most popular locations by zip code and plot them on map

In [40]:
top_zips = list(df_test.index[0:7].values)
top_zips

['20001', '20002', '20003', '20005', '20006', '20007', '20009']

In [41]:
#Get the latitude and longitude for these top 7 zip codes.
lat_list = [] 
long_list =[]
for zipCode in top_zips:
    address2 = zipCode
    geolocator2 = Nominatim(user_agent="dc_explorer")
    location2 = geolocator.geocode(address2 + " , United States")
    lat_list.append(location2.latitude)
    long_list.append(location2.longitude)

In [137]:
#Plot are zip code locations 
#venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Newtown Square

In [43]:
# add popular spots to the map as blue circle markers
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

for lat, lng, label in zip(lat_list, long_list, top_zips):
    folium.features.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        fill=True,
        color='green',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

## Now we have are reestraunt locations and need to generatre distance/time matrix 

In [44]:
#20006 ZIP CODE for walkthrough.
df_travel = pd.read_csv('/Users/thomasottotrankle/Desktop/projects/Coursera_Capstone/Travel_Times.csv')

print('Dataset downloaded and read into a pandas dataframe!')

Dataset downloaded and read into a pandas dataframe!


In [45]:
df_travel.head()

,Origin Movement ID,Origin Display Name,Destination Movement ID,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds)
0,168,"Washington, DC",1,"5400 Arnold Avenue Southwest, Southwest Washin...","1/1/2020 - 3/31/2020, Every day, Daily Average",1023,711,1471
1,168,"Washington, DC",2,"1400 Juniper Street Northwest, Northwest Washi...","1/1/2020 - 3/31/2020, Every day, Daily Average",1613,1297,2004
2,168,"Washington, DC",3,"4800 Canal Road Northwest, Northwest Washingto...","1/1/2020 - 3/31/2020, Every day, Daily Average",855,634,1151
3,168,"Washington, DC",4,"2700 Unicorn Lane Northwest, Northwest Washing...","1/1/2020 - 3/31/2020, Every day, Daily Average",1399,1129,1732
4,168,"Washington, DC",5,"4500 Q Place Northwest, Northwest Washington, ...","1/1/2020 - 3/31/2020, Every day, Daily Average",575,413,799


In [47]:
import json

In [46]:
# Opening JSON file 
f = open('washington_DC_censustracts.json',) 
  
# returns JSON object as  
# a dictionary 
dc_json = json.load(f) 
  
# Closing file 
f.close() 

In [48]:
dc_data = json_normalize(dc_json)

In [49]:
dc_data.columns

Index(['type', 'features'], dtype='object')

## The dataframe containing the JSON file has a lot of nested loops. After some experiementation, the end location can be found as followed/

In [115]:
#Latitude and longitude corresponding to Destination Movement ID 0
dc_data['features'][0][0]['geometry']['coordinates'][0][0][-1]

[-77.048009, 38.841266]

### Now lets add the latitutde and lingitude for every destination address in our df_travel

In [50]:
#rename column to iterate through
df_travel.rename(columns={"Destination Movement ID": "final_loc_id"}, inplace = True)

In [51]:
lat20006_list = []
lng20006_list = []

#Notice latitude and longitude are reversed
for i in df_travel.final_loc_id:
    lng20006_list.append(dc_data['features'][0][i-1]['geometry']['coordinates'][0][0][-1][0])
    lat20006_list.append(dc_data['features'][0][i-1]['geometry']['coordinates'][0][0][-1][1])

In [52]:
len(lat20006_list)

547

### add this list to our ccurent df_travel

In [53]:
df_travel['lat'] = lat20006_list
df_travel['lng'] = lng20006_list
df_travel

,Origin Movement ID,Origin Display Name,final_loc_id,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds),lat,lng
0,168,"Washington, DC",1,"5400 Arnold Avenue Southwest, Southwest Washin...","1/1/2020 - 3/31/2020, Every day, Daily Average",1023,711,1471,38.841266,-77.048009
1,168,"Washington, DC",2,"1400 Juniper Street Northwest, Northwest Washi...","1/1/2020 - 3/31/2020, Every day, Daily Average",1613,1297,2004,38.987145,-77.051623
2,168,"Washington, DC",3,"4800 Canal Road Northwest, Northwest Washingto...","1/1/2020 - 3/31/2020, Every day, Daily Average",855,634,1151,38.934350,-77.119751
3,168,"Washington, DC",4,"2700 Unicorn Lane Northwest, Northwest Washing...","1/1/2020 - 3/31/2020, Every day, Daily Average",1399,1129,1732,38.971786,-77.071647
4,168,"Washington, DC",5,"4500 Q Place Northwest, Northwest Washington, ...","1/1/2020 - 3/31/2020, Every day, Daily Average",575,413,799,38.911209,-77.100859
...,...,...,...,...,...,...,...,...,...,...
542,168,"Washington, DC",554,"1300 North Beauregard Street, Alexandria West,...","1/1/2020 - 3/31/2020, Every day, Daily Average",893,673,1183,38.825350,-77.131607
543,168,"Washington, DC",555,"5900 Atteentee Road, Springfield","1/1/2020 - 3/31/2020, Every day, Daily Average",1415,1104,1813,38.792764,-77.218782
544,168,"Washington, DC",556,"7800 Montvale Way, McLean","1/1/2020 - 3/31/2020, Every day, Daily Average",1002,804,1247,38.934171,-77.233023
545,168,"Washington, DC",557,"8400 Fenwood Drive, West Springfield","1/1/2020 - 3/31/2020, Every day, Daily Average",1554,1204,2005,38.789962,-77.241731


## Additionally, we will add a column for Mean Travel Time in minutes.

In [54]:
df_travel['Mean Travel Time (Minutes)'] = df_travel['Mean Travel Time (Seconds)']/60
df_travel

,Origin Movement ID,Origin Display Name,final_loc_id,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds),lat,lng,Mean Travel Time (Minutes)
0,168,"Washington, DC",1,"5400 Arnold Avenue Southwest, Southwest Washin...","1/1/2020 - 3/31/2020, Every day, Daily Average",1023,711,1471,38.841266,-77.048009,17.050000
1,168,"Washington, DC",2,"1400 Juniper Street Northwest, Northwest Washi...","1/1/2020 - 3/31/2020, Every day, Daily Average",1613,1297,2004,38.987145,-77.051623,26.883333
2,168,"Washington, DC",3,"4800 Canal Road Northwest, Northwest Washingto...","1/1/2020 - 3/31/2020, Every day, Daily Average",855,634,1151,38.934350,-77.119751,14.250000
3,168,"Washington, DC",4,"2700 Unicorn Lane Northwest, Northwest Washing...","1/1/2020 - 3/31/2020, Every day, Daily Average",1399,1129,1732,38.971786,-77.071647,23.316667
4,168,"Washington, DC",5,"4500 Q Place Northwest, Northwest Washington, ...","1/1/2020 - 3/31/2020, Every day, Daily Average",575,413,799,38.911209,-77.100859,9.583333
...,...,...,...,...,...,...,...,...,...,...,...
542,168,"Washington, DC",554,"1300 North Beauregard Street, Alexandria West,...","1/1/2020 - 3/31/2020, Every day, Daily Average",893,673,1183,38.825350,-77.131607,14.883333
543,168,"Washington, DC",555,"5900 Atteentee Road, Springfield","1/1/2020 - 3/31/2020, Every day, Daily Average",1415,1104,1813,38.792764,-77.218782,23.583333
544,168,"Washington, DC",556,"7800 Montvale Way, McLean","1/1/2020 - 3/31/2020, Every day, Daily Average",1002,804,1247,38.934171,-77.233023,16.700000
545,168,"Washington, DC",557,"8400 Fenwood Drive, West Springfield","1/1/2020 - 3/31/2020, Every day, Daily Average",1554,1204,2005,38.789962,-77.241731,25.900000


### Let's see how many location are less than 20 minutes away.

In [55]:
df_travel['Mean Travel Time (Minutes)'].values < 20.0

array([ True, False,  True, False,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True, False, False,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
        True,  True,  True,  True,  True, False,  True, False, False,
        True, False,  True, False, False, False, False, False, False,
        True, False,  True,  True,  True,  True,  True, False,  True,
       False, False,

### Drop all rows that dont meet the time constraint of 20min

In [56]:
df_travel.drop(df_travel[df_travel['Mean Travel Time (Minutes)'].values > 20.0].index, inplace=True)

In [57]:
df_travel

,Origin Movement ID,Origin Display Name,final_loc_id,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds),lat,lng,Mean Travel Time (Minutes)
0,168,"Washington, DC",1,"5400 Arnold Avenue Southwest, Southwest Washin...","1/1/2020 - 3/31/2020, Every day, Daily Average",1023,711,1471,38.841266,-77.048009,17.050000
2,168,"Washington, DC",3,"4800 Canal Road Northwest, Northwest Washingto...","1/1/2020 - 3/31/2020, Every day, Daily Average",855,634,1151,38.934350,-77.119751,14.250000
4,168,"Washington, DC",5,"4500 Q Place Northwest, Northwest Washington, ...","1/1/2020 - 3/31/2020, Every day, Daily Average",575,413,799,38.911209,-77.100859,9.583333
5,168,"Washington, DC",6,"Ridge Road Northwest, Northwest Washington, Wa...","1/1/2020 - 3/31/2020, Every day, Daily Average",1066,838,1355,38.954336,-77.069425,17.766667
6,168,"Washington, DC",7,"4500 Ohio Drive Southwest, Southwest Washingto...","1/1/2020 - 3/31/2020, Every day, Daily Average",237,95,589,38.891852,-77.064696,3.950000
...,...,...,...,...,...,...,...,...,...,...,...
539,168,"Washington, DC",551,"4700 Kenmore Avenue, Seminary Hill, Alexandria","1/1/2020 - 3/31/2020, Every day, Daily Average",912,672,1236,38.824118,-77.124376,15.200000
540,168,"Washington, DC",552,"5700 Reading Avenue, Alexandria West, Alexandria","1/1/2020 - 3/31/2020, Every day, Daily Average",1097,812,1480,38.832170,-77.135240,18.283333
541,168,"Washington, DC",553,"4900 Seminary Road, Alexandria West, Alexandria","1/1/2020 - 3/31/2020, Every day, Daily Average",847,637,1126,38.834282,-77.117275,14.116667
542,168,"Washington, DC",554,"1300 North Beauregard Street, Alexandria West,...","1/1/2020 - 3/31/2020, Every day, Daily Average",893,673,1183,38.825350,-77.131607,14.883333


In [92]:
df_travel.dtypes

level_0                                        int64
index                                          int64
Origin Movement ID                             int64
Origin Display Name                           object
Destination Movement ID                        int64
Destination Display Name                      object
Date Range                                    object
Mean Travel Time (Seconds)                     int64
Range - Lower Bound Travel Time (Seconds)      int64
Range - Upper Bound Travel Time (Seconds)      int64
lat                                          float64
lng                                          float64
Mean Travel Time (Minutes)                   float64
categories                                    object
dtype: object

In [82]:
df_rated.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
city                 object
state                object
country              object
formattedAddress     object
neighborhood         object
id                   object
Rating              float64
dtype: object

In [59]:
la = list(df_travel.lat)
lo = list(df_travel.lng)

In [105]:
location

Location(Washington, District of Columbia, 20500, United States of America, (38.8949855, -77.0365708, 0.0))

In [60]:
map20006 = folium.Map(location=[latitude,longitude],zoom_start=11)

# Define our key point when map open it
for la1,lo1 in zip(la,lo):
    folium.Marker(
        [la1,lo1]
        
    ).add_to(map20006)
# Plotting 

map20006

### Create a dictionary to hold target variable results.

In [61]:
score_list = {}
score = dict.fromkeys(set(top_zips),[])

In [62]:
score

{'20005': [],
 '20002': [],
 '20003': [],
 '20007': [],
 '20001': [],
 '20009': [],
 '20006': []}

### Add score for 20006 zip code

In [63]:
score['20006'] = len(df_travel)
score

{'20005': [],
 '20002': [],
 '20003': [],
 '20007': [],
 '20001': [],
 '20009': [],
 '20006': 231}

## We repeat this process for the remaing starting points 

In [64]:
keys = ['20001','20002', '20003', '20005', '20007', '20009']

In [65]:
for i in range(len(keys)):  
    file = '/Users/thomasottotrankle/Desktop/projects/Coursera_Capstone/Travel_Times' + keys[i] + '.csv'
    df_travel = pd.read_csv(file)
    df_travel.rename(columns={"Destination Movement ID": "final_loc_id"}, inplace = True)
    lat_list = []
    lng_list = []

    for j in df_travel.final_loc_id:
        lng_list.append(dc_data['features'][0][j-1]['geometry']['coordinates'][0][0][-1][0])
        lat_list.append(dc_data['features'][0][j-1]['geometry']['coordinates'][0][0][-1][1])
    df_travel['lat'] = lat_list #same random final destination coordinates with every top zip code
    df_travel['lng'] = lng_list   
    df_travel['Mean Travel Time (Minutes)'] = df_travel['Mean Travel Time (Seconds)']/60
    df_travel.drop(df_travel[df_travel['Mean Travel Time (Minutes)'].values > 20.0].index, inplace=True)
    score[keys[i]] = len(df_travel)
    print(score)

    map = folium.Map(location=[latitude,longitude],zoom_start=11)

    # Define our key point when map open it
    for la1,lo1 in zip(df_travel.lat,df_travel.lng):
        folium.Marker(
            [la1,lo1]).add_to(map)
    # Plotting 
    display(map)

{'20005': [], '20002': [], '20003': [], '20007': [], '20001': 202, '20009': [], '20006': 231}


{'20005': [], '20002': 171, '20003': [], '20007': [], '20001': 202, '20009': [], '20006': 231}


{'20005': [], '20002': 171, '20003': 235, '20007': [], '20001': 202, '20009': [], '20006': 231}


{'20005': 200, '20002': 171, '20003': 235, '20007': [], '20001': 202, '20009': [], '20006': 231}


{'20005': 200, '20002': 171, '20003': 235, '20007': 132, '20001': 202, '20009': [], '20006': 231}


{'20005': 200, '20002': 171, '20003': 235, '20007': 132, '20001': 202, '20009': 148, '20006': 231}


In [66]:
print(score)

{'20005': 200, '20002': 171, '20003': 235, '20007': 132, '20001': 202, '20009': 148, '20006': 231}


In [71]:
max(score, key=score.get)


'20003'

## 20003 is the best zip code to open a new restraunt